<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/2.0-la-tj-ak-ensemble_baseline_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'

# Switch to TF 1.x and navigate to the directory
%tensorflow_version 1.x
!pwd
import os
os.chdir('cs231n-emotiw')
!pwd

# Install required packages 
!pip install -r 'requirements-predictions.txt'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Cloning into 'cs231n-emotiw'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 779 (delta 51), reused 37 (delta 17), pack-reused 683
Receiving objects: 100% (779/779), 176.99 MiB | 37.36 MiB/s, done.
Resolving deltas: 100% (459/459), done.
TensorFlow 1.x selected.
/content
/content/cs231n-emotiw
     |████████████████████████████████| 389kB 5.4MB/s 
     |█████

#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [2]:
!wget https://storage.googleapis.com/cs231n-emotiw/data/train-full.zip .
!wget https://storage.googleapis.com/cs231n-emotiw/data/val-full.zip .

!unzip "train-full.zip"
!unzip "val-full.zip"

--2020-06-02 16:12:34--  https://storage.googleapis.com/cs231n-emotiw/data/train-full.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 2607:f8b0:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2732428104 (2.5G) [application/zip]
Saving to: ‘train-full.zip’

train-full.zip      100%[===================>]   2.54G  31.8MB/s    in 48s     

2020-06-02 16:13:22 (54.1 MB/s) - ‘train-full.zip’ saved [2732428104/2732428104]

--2020-06-02 16:13:22--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2020-06-02 16:13:22--
Total wall clock time: 48s
Downloaded: 1 files, 2.5G in 48s (54.1 MB/s)
--2020-06-02 16:13:28--  https://storage.googleapis.com/cs231n-emotiw/data/val-full.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 2607:f8b0:400c:c06

In [3]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

# The pre-built OpenPose library contains shared library files that need to be manually linked
import os
orig_lib_path = os.environ['LD_LIBRARY_PATH']
os.environ["LD_LIBRARY_PATH"] = f"{orig_lib_path}:{os.getcwd()}/openpose/build/src/openpose/:{os.getcwd()}/openpose/build/caffe/lib/"



Selecting previously unselected package libgflags2.2.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

#### Preprocess Files

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [4]:
!wget https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt

--2020-06-02 16:15:44--  https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 2607:f8b0:400c:c0c::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21653 (21K) [text/plain]
Saving to: ‘Train_labels.txt’

Train_labels.txt    100%[===================>]  21.15K  --.-KB/s    in 0s      

2020-06-02 16:15:44 (127 MB/s) - ‘Train_labels.txt’ saved [21653/21653]

--2020-06-02 16:15:45--  https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 2607:f8b0:400c:c15::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6265 (6.1K) [text/plain]
Saving to: ‘Val_labels.txt’

Val_labels.txt      100%[===================>]  

In [5]:
!ls

 data		   README.md				       train-full.zip
 LICENSE	   reports				       Train_labels.txt
 __MACOSX	   requirements-predictions.txt		       Val
 models		   requirements.txt			       val-full.zip
 notebooks	  'Screen Shot 2020-05-26 at 8.53.32 PM.png'   Val_labels.txt
 openpose	   src
 openpose.tar.gz   Train


In [0]:
from src.preprocessors.preprocess_all_modes import preprocess
from src.preprocessors.pose_preprocessor import PosePreprocessor

# print("Starting to preprocess train data")
# preprocess(video_folder="train-tiny.zip", label_file="Train_labels.txt", local_base_path="train-tiny")

# print("Starting to preprocess val data")
# preprocess(video_folder="val-tiny.zip", label_file="Val_labels.txt", local_base_path="val-tiny")

# print("Starting to preprocess test data")
# preprocess(video_folder="test-tiny.zip", local_base_path="test-tiny")

print("Starting to preprocess train data")
preprocess(video_folder="Train", label_file="Train_labels.txt", local_base_path="train-full")

!cp train-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"

print("Starting to preprocess val data")
preprocess(video_folder="Val", label_file="Val_labels.txt", local_base_path="val-full")

!cp val-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"

Using TensorFlow backend.


Streaming output truncated to the last 5000 lines.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_18.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_19.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_2.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_3.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_4.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_5.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_6.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Reading file 29_7.mp4extracted_audio.wav ...
Moviepy - Running:
>>> "+ " ".join(

In [0]:
!ls train-full-frames_tmp

Train


In [0]:
# Remove the openpose folder as it is no longer required
!rm -rf openpose/

In [0]:
!ls

cs231n-emotiw  sample_data


In [0]:


!cp train-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"
!cp val-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"
# !cp test-full-*.zip "/content/drive/My Drive/cs231n-project/datasets/emotiw"